# Assignment 4: Training Transformers in PyTorch

*Author:* Thomas Adler

*Copyright statement:* This  material,  no  matter  whether  in  printed  or  electronic  form,  may  be  used  for  personal  and non-commercial educational use only.  Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.

In this assignment we will implement and train a small transformer model and compare it to the LSTM in the previous assignment. 

## Exercise 1: Causal Self-Attention

Write a class named `CausalSelfAttention` that derives from `nn.Module` and whose `__init__` method takes (apart from the trivial `self`) one argument `hidden_size`. Implement a method `forward` that takes an input sequence `x` of shape $(N, T, D)$ (where $N$ is batch size, $T$ is sequence length, $D$ is hidden size) and performs scaled dot-product self-attention, i.e., 
$$
Y = \operatorname{softmax}\left(\frac{1}{\sqrt{D}} Q K^\top\right) V,
$$
where $Q = X W_Q$ and $K = X W_K$ and $V = X W_V$ and $X \in \mathbb{R}^{T \times D}$ and $W_Q, W_K, W_V \in \mathbb{R}^{D \times D}$ and softmax is applied in a row-wise manner and neglecting bias units. 
It is called self-attention because $Q, K, V$ are all computed from the same input $X$, which hence attends to itself. 

To have the attention be *causal* we need to make sure that we do not allow peeks into the future. That is, the output at time $t$ must be a function of the input at times $1, \dots, t$ but no further. The score matrix $E = \frac{1}{\sqrt{D}} Q K^\top$ has a shape of $T \times T$ and the entry $e_{ij}$ measures how strong the query at time $i$ attends to the key at time $j$. Therefore, positions where $j > i$ constitute peeks into the future and we have to set the corresponding attention values (i.e., the softmax-activated score) to zero. We can do that by setting the corresponding score to `float('-inf')`, which has the advantage that the normalization is adjusted automatically by the softmax. 

In [117]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import time

########## YOUR SOLUTION HERE ##########
class CausalSelfAttention(nn.Module):
    def __init__(self, hidden_size):
        super(CausalSelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.Q = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)
        self.K = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)
        self.V = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)

    def forward(self, x):

        N, T, D = x.shape
        Q = x @ self.Q 
        K = x @ self.K   
        V = x @ self.V
       
        scores = (Q @ K.transpose(-2, -1)) / math.sqrt(D)
        causal_mask = torch.triu(torch.ones(T, T, device=x.device), diagonal=1).bool()
        scores.masked_fill_(causal_mask, float('-inf'))
        attention_weights = F.softmax(scores, dim=-1)
        Y = attention_weights @ V 

        return Y



## Exercise 2: Multi-Head Attention

Write a class `MultiHeadCausalSelfAttention` that derives from `nn.Module` and extends the functionality of `CausalSelfAttention` from the previous exercise. 
The `__init__` method takes arguments `hidden_size, n_head, dropout`. `n_head` specifies the number of attention heads and `dropout` specifies the intensity for the dropout layers. 
The `forward` method should split the hidden dimension of the pre-activations (i.e., $Q, K, V$) in `n_head` equally sized parts and perform attention to these parts in parallel. 
Apply the first dropout layer direcly after the softmax. 
After the multiplication of the scores with the values, recombine the output of the distinct attention heads back into a single hidden dimension of size $D$, i.e., the resulting shape should be the shape of the input. 
Then perform an additional output projection again resulting in a hidden dimension of $D$. 
Finally, apply the second dropout layer after the output projection. 

In [119]:
########## YOUR SOLUTION HERE ##########
class MultiHeadCausalSelfAttention(nn.Module):
    def __init__(self, hidden_size, n_head, dropout):
        super(MultiHeadCausalSelfAttention, self).__init__()
        assert hidden_size % n_head == 0, "hidden_size must be divisible by n_head"

        self.hidden_size = hidden_size
        self.n_head = n_head
        self.head_dim = hidden_size // n_head
        self.dropout = nn.Dropout(dropout)

        # Weight matrices for Q, K, V
        self.Q = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)
        self.K = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)
        self.V = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)
        self.W_O = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)

    def forward(self, x):
        
        N, T, D = x.shape
        q = x @ self.Q
        k = x @ self.K
        v = x @ self.V
        Q = q.view(N, T, self.n_head, self.head_dim).transpose(1, 2)
        K = k.view(N, T, self.n_head, self.head_dim).transpose(1, 2)
        V = v.view(N, T, self.n_head, self.head_dim).transpose(1, 2)
       # print("Q.transpose(1, 2):",Q.transpose(1, 2).shape)
        #print("K.transpose(1, 3):",K.transpose(1, 3).shape)
        scores = (Q @ K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        causal_mask = torch.triu(torch.ones(T, T, device=x.device), diagonal=1).bool()
        scores.masked_fill_(causal_mask, float('-inf'))
        att_w = F.softmax(scores, dim=-1)
        att_w = self.dropout(att_w)
        Y = att_w @ V
        Y = Y.transpose(1, 2).contiguous().view(N, T, D)
        Y = Y @ self.W_O
        Y = self.dropout(Y)

        return Y



## Exercise 3: Multi-Layer Perceptron

Write a class `MLP` that derives from `nn.Module` and whose `__init__` method takes two arguments: `hidden_size` and `dropout`. 
It should implement a 2-layer feedforward network with `hidden_size` inputs, `4*hidden_size` hiddens, and `hidden_size` outputs. 
It should apply the GELU activation function to the hiddens and dropout to the outputs. 

In [121]:
########## YOUR SOLUTION HERE ##########
class MLP(nn.Module):
    def __init__(self, hidden_size, dropout):
        super(MLP, self).__init__()
        self.hidden_size = hidden_size

        self.f1 = nn.Linear(hidden_size, 4 * hidden_size)
        self.f2 = nn.Linear(4 * hidden_size, hidden_size)
        self.gelu = nn.GELU()
        self.dpt = nn.Dropout(dropout)

    def forward(self, x):
        x = self.f1(x)  
        x = self.gelu(x) 
        x = self.dpt(x)  
        x = self.f2(x)  
        x = self.dpt(x)  
        return x



## Exercise 4: Block

Write a class `Block` that derives from `nn.Module` and whose `__init__` method takes arguments `hidden_size, n_head, dropout`. 
It should apply `nn.LayerNorm`, `CausalMultiHeadSelfAttention`, `nn.LayerNorm`, `MLP` in that order and feature residual connections from the input to the output of `CausalMultiHeadSelfAttention` and from there to the output of `MLP`. 

In [123]:
########## YOUR SOLUTION HERE ##########
class Block(nn.Module):
    def __init__(self, hidden_size, n_head, dropout):
        super(Block, self).__init__()
        self.hidden_size = hidden_size
        self.ln1 = nn.LayerNorm(hidden_size)
        self.ln2 = nn.LayerNorm(hidden_size)

    def forward(self, x):
        att = MultiHeadCausalSelfAttention(hidden_size, n_head, dropout)
        mlp = MLP(hidden_size, dropout)
        x = x + att(self.ln1(x))
        x = x + mlp(self.ln2(x))
        return x


## Exercise 5: GPT

Write a class `GPT` that derives from `nn.Module` and whose `__init__` method takes arguments `vocab_size, context_size, hidden_size, n_layer, n_head, dropout`. 
The `forward` method should take two arguments `x, y` representing sequences of input and target tokens, respectively, both of which have type `torch.long` and shape ($N$, $T$), and returns logits and loss as a tuple. 
The `GPT` module should feature two `nn.Embedding` layers, one for token embeddings and one for positional embedding, i.e., it should embed the position of the corresponding token within the input sequence. 
The positional embedding is necessary for the Transformer to determine the order of its inputs. 
Add the two embeddings and apply a dropout layer. 
Next, apply `n_layers` layers of `Block`s followed by a `nn.LayerNorm` and a `nn.Linear` (without bias) mapping to an output dimension of `vocab_size`. 
Finally, apply the cross-entropy loss function to the logits. 
To save some parameters, apply weight tying between the token embedding layer and the output layer, i.e., they should use the same weights. 
Initialize all weights using a normal distribution with a mean of zero and a standard deviation of 0.02 (except for the output layers of the `MLP`s use $0.02/\sqrt{2 * \mathtt{n\_layer}}$) and all biases to zero. 
Use the argument `dropout` as intensity for all dropout layers in the network. 

In [96]:
########## YOUR SOLUTION HERE ##########
class GPT(nn.Module):
    def __init__(self, vocab_size, context_size, hidden_size, n_layer, n_head, dropout):
        super(GPT, self).__init__()

        self.token_embed = nn.Embedding(vocab_size, hidden_size)
        self.position_embed = nn.Embedding(context_size, hidden_size)
        self.ln = nn.LayerNorm(hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.blocks = nn.ModuleList([Block(hidden_size, n_head, dropout) for _ in range(n_layer)])
        self.fc.weight = self.token_embed.weight
        self._init_weights(n_layer)

    def _init_weights(self, n_layer):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=0.02 / math.sqrt(2 * n_layer))
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, x, y):
        N, T = x.shape
        token_embeds = self.token_embed(x)  
        position_embeds = self.position_embed(torch.arange(T, device=x.device))  
        position_embeds = position_embeds.unsqueeze(0).expand(N, T, -1)
        x = self.dropout(token_embeds + position_embeds)
        
        for block in self.blocks:
            x = block(x)

        x = self.ln(x)
        #x = nn.LayerNorm(hidden_size)
        logs = self.fc(x)
        #logs = nn.Linear(hidden_size, vocab_size, bias=False)
        loss = F.cross_entropy(logs.view(-1, logs.size(-1)), y.view(-1), ignore_index=-1)

        return logs, loss


Logits shape: torch.Size([4, 128, 10000])
Loss: 9.222928047180176


## Exercise 6: Optimizer

Add a method `configure_optimizers` to the class `GPT` that takes arguments `weight_decay, learning_rate, betas`. 
Divide the model parameters into two groups. 
The first group consists of all parameters with at least 2 dimensions, e.g., weight/embedding matrices and uses a decay of `weight_decay`. 
The second group consists of all other parameters, e.g., biases and layer norms, and does not use weight decay.
Construct and return a `torch.optim.AdamW` optimizer with `learning_rate` and `betas` that operates on these two parameter groups. 

In [98]:
########## YOUR SOLUTION HERE ##########
def configure_optimizers(self, weight_decay, learning_rate, betas):
    decay_params = []
    no_decay_params = []

    for name, param in self.named_parameters():
        if param.ndim >= 2: 
            decay_params.append(param)
        else:  
            no_decay_params.append(param)

    optimizer = torch.optim.AdamW([{"params": decay_params, "weight_decay": weight_decay}, {"params": no_decay_params, "weight_decay": 0.0}, ], lr=learning_rate, betas=betas)
    return optimizer

GPT.configure_optimizers = configure_optimizers

## Exercise 7: Training

In the code cell below you find some globals, helper functions, and boilerplate code. Extend the given code by a training loop that 
* stops after `max_iters` iterations
* applies the learning rate schedule implemented in `get_lr`
* applies gradient clipping at `grad_clip` using `torch.nn.utils.clip_grad_norm_`
* accumulates gradients for `gradient_accumulation_steps` batches before each weight update
* logs the training loss and learning rate every `log_interval` iterations
* evaluates (and potentially checkpoints) the model using `estimate_loss` every `eval_iters` iterations.

The provided hyperparameter values should be a good guess for training a tiny model on CPU but feel free to experiment with them as you please. In particular, if you have a GPU available, you can try to scale things up a bit. 

In [132]:
eval_interval = 250 # validate model every .. iterations
log_interval = 10 # log training loss every .. iterations
eval_iters = 20 # number of batches for loss estimation
gradient_accumulation_steps = 5 * 8 # used to simulate larger training batch sizes
batch_size = 12 # if gradient_accumulation_steps > 1, this is the micro-batch size
context_size = 64 # sequence length
vocab = 'abcdefghijklmnopqrstuvwxyz0123456789 .!?' # vocabulary
vocab_size = len(vocab) # 40
n_layer = 4 # number of layers
n_head = 4 # number of attention heads
hidden_size = 128 # layer size
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
learning_rate = 1e-3 # max learning rate
max_iters = 500 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9 # for AdamW
beta2 = 0.99 # for AdamW
grad_clip = 1.0 # clip gradients at this value, or disable with 0.0
warmup_iters = 100 # how many steps to warm up for
min_lr = 1e-4 # minimum learning rate, usually ~= learning_rate/10

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > max_iters, return min learning rate
    if it > max_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (max_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

def load_data(split):
    import re
    
    with open(f'trump_{split}.txt', 'r', encoding='utf-8') as f:
        text = f.read()
    
    text = text.lower() # convert to lower case
    text = re.sub('[^a-z0-9 .!?]', ' ', text) # replace all unknown chars with ' '
    text = re.sub(' +', ' ', text) # reduce multiple blanks to one
    text = [vocab.index(t) for t in text]
    text = torch.tensor(text, dtype=torch.long)
    return text
    
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_size, (batch_size,))
    x = torch.stack([data[i:i+context_size] for i in ix])
    y = torch.stack([data[i+1:i+1+context_size] for i in ix])
    return x, y

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# data, model, optimizer, etc. 
train_data = load_data('train')
val_data = load_data('val')
model = GPT(vocab_size, context_size, hidden_size, n_layer, n_head, dropout)
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2))
iter_num = 0
best_val_loss = 1e9
X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()

########## YOUR SOLUTION HERE ##########

for iter_num in range(max_iters):
    comb_loss = 0.0
    optimizer.zero_grad()

    for step in range(gradient_accumulation_steps):
        X, Y = get_batch('train')
       # X, Y = X.cuda(), Y.cuda()
      #  model = model.cuda()
        logs, loss = model(X, Y)
        loss = loss / gradient_accumulation_steps 
        loss.backward()
        comb_loss += loss.item()
    if grad_clip > 0.0:
        par = model.parameters()
        torch.nn.utils.clip_grad_norm_(par, grad_clip)

    optimizer.step()

    lr = get_lr(iter_num)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    if iter_num % log_interval == 0:
        mean_loss = comb_loss / gradient_accumulation_steps
      #  print(f"Iter {iter_num}/{max_iters}, Loss: {mean_loss:.4f}, LR: {lr:.6e}")
        print(f"Iter {iter_num}/{max_iters}, Loss: {mean_loss:.4f}")

    if iter_num % eval_interval == 0:
        eval_loss = estimate_loss()
        #print(f"[{iter_num] Train Loss: {mean_loss['train']:.4f}, Val Loss: {mean_loss['val']:.4f}")
        print(f"[{iter_num}] Train Loss: {eval_loss['train']:.4f}, Val Loss: {eval_loss['val']:.4f}")

        if eval_loss['val'] < best_val_loss:
            best_val_loss = eval_loss['val']
            torch.save(model.state_dict(), 'best_model.pt')
            print(f"best model saved with val loss: {best_val_loss:.4f}")

print("Training complete.")
print(f"Best validation loss: {best_val_loss:.4f}")

Iter 0/500, Loss: 0.0923
[0] Train Loss: 3.6924, Val Loss: 3.6941
best model saved with val loss: 3.6941
Iter 10/500, Loss: 0.0923
Iter 20/500, Loss: 0.0923
Iter 30/500, Loss: 0.0922
Iter 40/500, Loss: 0.0921
Iter 50/500, Loss: 0.0920
Iter 60/500, Loss: 0.0913
Iter 70/500, Loss: 0.0909
Iter 80/500, Loss: 0.0893
Iter 90/500, Loss: 0.0858
Iter 100/500, Loss: 0.0824
Iter 110/500, Loss: 0.0799
Iter 120/500, Loss: 0.0770
Iter 130/500, Loss: 0.0748
Iter 140/500, Loss: 0.0745
Iter 150/500, Loss: 0.0743
Iter 160/500, Loss: 0.0738
Iter 170/500, Loss: 0.0734
Iter 180/500, Loss: 0.0732
Iter 190/500, Loss: 0.0734
Iter 200/500, Loss: 0.0735
Iter 210/500, Loss: 0.0731
Iter 220/500, Loss: 0.0729
Iter 230/500, Loss: 0.0730
Iter 240/500, Loss: 0.0730
Iter 250/500, Loss: 0.0728
[250] Train Loss: 2.9159, Val Loss: 2.9280
best model saved with val loss: 2.9280
Iter 260/500, Loss: 0.0733
Iter 270/500, Loss: 0.0730
Iter 280/500, Loss: 0.0729
Iter 290/500, Loss: 0.0730
Iter 300/500, Loss: 0.0727
Iter 310/500

## Exercise 8: Inference

Add a method `generate` to the class `GPT` that takes arguments `x, max_new_tokens, temperature=1.0`. 
The method should take a batch of token sequences `x`, which it should extend by `max_new_tokens` new tokens generated by the model. 
Once you have computed the logits for the next token, divide them by `temperature` before applying the softmax. 
After applying the softmax, sample the next token from the resulting categorical distribution. 
Try out different values for `temperature` and compare the results to those from the previous assignment. 

In [134]:
########## YOUR SOLUTION HERE ##########
def generate(self, x, max_new_tokens, temperature=1.0):

        model.eval()
        generated = x.clone()

        for _ in range(max_new_tokens):
            in_seq = generated[:, -context_size:]
            logs, _ = self(in_seq, None)
            logs = logs[:, -1, :] / temperature
            probs = torch.softmax(logs, dim=-1)
            new_token = torch.multinomial(probs, num_samples=1)
            generated = torch.cat((generated, new_token), dim=1)

        model.train()
        return generated